In [290]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async, RateLimiter, send_message_open_router
from pprint import pprint

rate_limiter = RateLimiter(13, 14)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [291]:
base_message_algorithm = """Вы тактичный и вежливый ИИ-ассистент которого зовут Аика, вы помогаете ученику самостоятельно собрать алгоритм по распорядку дня для улучшения здоровья. 
Вам по очереди дана json карточка ученика вместе с инструкцииями. Вам необходимо построить диалог таким образом, чтобы заполнить все обязательные поля в карточке. 
Вместо того, чтобы давать готовые решения, старайтесь использовать сократовский метод, задавая наводящие вопросы, чтобы помочь ученику самостоятельно создать свой собственный алгоритм дня.
Начните с первого вопроса про имя, дождись ответа пользователя, и далее следуйте по плану, задавая вопросы по одному, чтобы заполнить все незаполненные поля в карточке - спрашивай только 1 вопрос в одном сообщении. 
Пиши только сообщения пользователю, как будто бы ты общаешься с ним через мессенджер. Будь вежливой, счастливой и остроумной. Не забывай уместно благодорить ученика за ответы.

Вот карточка ученика:
<student_card>
{$STUDENT_CARD}
</student_card>
"""

student_card = {
  "description of card": "карточка о здоровье, хранических заболеваниях, выбранном лечении",
  "Имя и возраст": {"value": ""},
  "Цели по здоровью": {"value": ""},
  "Промокод": {"value": "", "instruction": "Спросите промокод, который он получил в девятом уроке. Он открывает доступ к формированию алгоритма. Без промокода доступ к алгоритму будет закрыт и Аика выйдет из этого сценария"},
  "Впечатления о курсе": {"value": ""},
  "Артериальное давление": {"value": "", "instruction": "Спросить про точное значение артериального давления, а также какой тип соответствует его давлению: Гипотоник (менее 110/70), нормотоник (110/70 - 130/90) или гипертоник (более 130/90)."},
  "Тип сокращения желчного пузыря": {"value": "", "instruction": "Спросить, сделано ли УЗИ и какой тип сокращения желчного пузыря. Возможные варианты: нормокинетический, гипо, гипер, неизвестно, желчный удален."},
  "Трава желчегонная": {"value": "", "instruction": "Спросить какую он подобрал желчегонную траву. Убедиться, что выбрана трава соответствует типу артериального давления. Гиперторику подходят только кукурузные рыльца которые уменьшают давление. Гипотонику подходит только пижма которая увеличивает давление, а нормотонику подходит тысячелистник который не изменяет давление."},
  "Желчегонная гимнастика": {"value": "", "instruction": "Спросить о наличии камней в желчном пузыре, если их нет, то предложить выполнять желчегонную гимнастику. Урок по ссылке: https://goo.su/WlFpAPi"},
  "Уровень витамина Д в организме": {"value": "", "instruction": "Спросить сдавал ли он анализы и в каких количествах планирует принимать."},
  "Доза витамина Д": {"value": "", "instruction": "Спросить в каких количествах планирует принимать. Если он не уверен, то посоветовать обратиться к Куратору Марии - ник в телеграме @mariasmirnova03. Стандартная доза 4000 МЕ"},
  "Время принятия лимфатического душа": {"value": "", "instruction": "Предложить технику лимфатического душа, спросить предпочтительное время принятия душа (утро или вечер)."},
  "Результаты копрограммы": {"value": "", "instruction": "Спросить о результатах копрограммы, если они имеются."},
  "Хронические заболевания": {"value": {}, "instruction": "Наличие заболеваний или противопоказаний"}
}


extract_student_health_data_prompt = """Вам представлен диалог между ИИ-ассистентом с именем Model и Пользователем User. 
Внимательно прочитайте его, и поймите где спрашивает Model, а где отвечает User.
Ваша задача - извлечь релевантную информацию о юзере из фрагмента диалога между моделью и юзером.
Особо важно извлечь те поля, которые представлены в карточке клиента. Кроме этого, нужно извлекать другую важную информацию о юзере, если он ее предоставляет.

Формат диалога:
Model: [Вопрос ассистента]
User: [Ответ пользователя]

Схема карточки здоровья ученика, поля в которой необходимо заполнить:
<student_card>
{$STUDENT_CARD}
</student_card>

Извлеките и представьте информацию в формате питоновского словаря. 
Пример:
```json
{"Артериальное давление": "нормотоник 120/70"}
```

Фрагмент диалога:
<dialog>
{$DIALOG}
</dialog>
"""



In [292]:
from llm_api_calls import send_message_to_gemini_async


async def generate_readable_history_from_end(messages, num_pairs):
    """
    Generates a readable history from the end of a list of messages, ensuring that the history
    starts with a message from the model and ends with a message from the user, from the last messages.
    It includes a specified number of model-user message pairs from the end.

    Parameters:
    - messages (list of dicts): The messages in the conversation.
    - num_pairs (int): The number of model-user pairs to include in the output, counting from the end.

    Returns:
    - str: A string representing the readable history.
    """

    # Reverse the list to start from the end
    messages_reversed = list(reversed(messages))

    # Filter messages to get only those with 'model' and 'user' roles
    filtered_messages = [msg for msg in messages_reversed if msg['role'] in ['model', 'user']]

    # Ensure the last message (first in the reversed list) is from the 'user'
    if filtered_messages and filtered_messages[0]['role'] != 'user':
        # If the last message is not from 'user', try to adjust by removing the first message in the reversed list
        filtered_messages = filtered_messages[1:]

    # Limit the number of pairs, considering we're working in reverse
    limited_messages = []
    for i in range(0, min(len(filtered_messages), num_pairs * 2), 2):
        # Check if there's a subsequent (in reverse) model message to form a complete pair
        if i + 1 < len(filtered_messages) and filtered_messages[i]['role'] == 'user' and filtered_messages[i + 1]['role'] == 'model':
            limited_messages.extend(filtered_messages[i:i+2])

    # Generate the readable history from the limited messages, then reverse it to correct the order
    readable_history = '\n'.join([f"{item['role'].title()}: {' '.join(item['parts'])}" for item in reversed(limited_messages)])
    
    return readable_history


async def send_message(prompted_message, history, usage_history=[]):
    history.append({"role": "user", "content": prompted_message})

    # response = send_message_open_router(history, rate_limiter=rate_limiter)
    response = await send_message_to_gemini_async(history, rate_limiter=rate_limiter, generation_params={"temperature": 0.2, "top_k":3})
    input_tokens = response.get('input_tokens')
    output_tokens = response.get('output_tokens')
    answer_text = response.get('text_response')

    history.append({"role": "model", "parts": [answer_text]})
    if len(usage_history) > 0:
        conversation_tokens = usage_history[-1].get("conversation_tokens", 0) + input_tokens + output_tokens
    else:
        conversation_tokens = input_tokens + output_tokens

    usage_history.append({"input_tokens":input_tokens, "output_tokens":output_tokens, "conversation_tokens":conversation_tokens})

    print(usage_history[-1])
    print('\n\nUSER:\n', prompted_message)
    print('\n\MODEL:\n', answer_text)


async def add_to_student_card(readable_history, extraction_history=[], usage_history=[]):

    extract_student_health_data_prompted = extract_student_health_data_prompt.replace("$DIALOG", readable_history).replace("$STUDENT_CARD", str(student_card))
    extraction_history = [{"role": "user", "parts": [extract_student_health_data_prompted]}]
    response = await send_message_to_gemini_async(extraction_history, rate_limiter=rate_limiter, generation_params={"temperature": 0, "top_k":1})

    input_tokens = response.get('input_tokens')
    output_tokens = response.get('output_tokens')
    answer_text = response.get('text_response')

    try:
        new_info = json.loads(answer_text.strip('```json').strip('```'))
        student_card.update(new_info)
        print('\n\n NEW INFO', new_info)
    except:
        print('\n\n NEW INFO не найдена', response.get("text_response"))

    if len(usage_history) > 0:
        conversation_tokens = usage_history[-1].get("conversation_tokens", 0) + input_tokens + output_tokens
    else:
        conversation_tokens = input_tokens + output_tokens
    usage_history.append({"input_tokens":input_tokens, "output_tokens":output_tokens, "conversation_tokens":conversation_tokens})
    print(usage_history[-1])
    return response


message_history = []
usage_history = []
prompted_message = base_message_algorithm.replace("$STUDENT_CARD", str(student_card))
response = await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 869, 'output_tokens': 47, 'conversation_tokens': 916}


USER:
 Вы тактичный и вежливый ИИ-ассистент которого зовут Аика, вы помогаете ученику самостоятельно собрать алгоритм по распорядку дня для улучшения здоровья. 
Вам по очереди дана json карточка ученика вместе с инструкцииями. Вам необходимо построить диалог таким образом, чтобы заполнить все обязательные поля в карточке. 
Вместо того, чтобы давать готовые решения, старайтесь использовать сократовский метод, задавая наводящие вопросы, чтобы помочь ученику самостоятельно создать свой собственный алгоритм дня.
Начните с первого вопроса про имя, дождись ответа пользователя, и далее следуйте по плану, задавая вопросы по одному, чтобы заполнить все незаполненные поля в карточке - спрашивай только 1 вопрос в одном сообщении. 
Пиши только сообщения пользователю, как будто бы ты общаешься с ним через мессенджер. Будь вежливой, счастливой и остроумной. Не забывай уместно благодорить ученика за ответы.

Вот карточка ученика

In [288]:
prompted_message = 'Меня зовут Николай, 29 лет. А тебе?'
await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 12, 'output_tokens': 42, 'conversation_tokens': 636}


USER:
 Меня зовут Николай, 29 лет. А тебе?

\MODEL:
 Мне не нужно имя, я же ИИ-ассистент. :) Давай вернемся к твоему алгоритму. Каковы твои цели по здоровью? Чего ты хочешь достичь?


In [289]:
message_history

[{'role': 'user',
  'parts': ["Вы тактичный и вежливый ИИ-ассистент которого зовут Аика, вы помогаете ученику самостоятельно собрать алгоритм по распорядку дня для улучшения здоровья. \nВам по очереди дана json карточка ученика вместе с инструкцииями. Вам необходимо построить диалог таким образом, чтобы заполнить все обязательные поля в карточке. \nВместо того, чтобы давать готовые решения, старайтесь использовать сократовский метод, задавая наводящие вопросы, чтобы помочь ученику самостоятельно создать свой собственный алгоритм дня.\nНачните с первого вопроса про имя, дождись ответа пользователя, и далее следуйте по плану, задавая вопросы по одному, чтобы заполнить все незаполненные поля в карточке - спрашивай только 1 вопрос в одном сообщении. \nПиши только сообщения пользователю, как будто бы ты общаешься с ним через мессенджер. Будь вежливой, счастливой и остроумной. Не забывай уместно благодорить ученика за ответы.\n\nВот карточка ученика:\n<student_card>\n{{'description of card':

In [266]:
prompted_message = 'Я хочу наладить сон, быть более энергичным, меньше болеть. А ты?'
await send_message(prompted_message, message_history, usage_history)
readable_history = await generate_readable_history_from_end(message_history, 2)
response_new_info = await add_to_student_card(readable_history, extraction_history=[], usage_history=[])

{'input_tokens': 20, 'output_tokens': 36, 'conversation_tokens': 1042}


USER:
 Я хочу наладить сон, быть более энергичным, меньше болеть. А ты?

\MODEL:
 У тебя замечательные цели, Николай! А есть ли у тебя промокод, который ты получил в девятом уроке? Он откроет нам доступ к формированию алгоритма.


 NEW INFO {'Имя и возраст': {'value': 'Николай, 29 лет'}, 'Цели по здоровью': {'value': 'Наладить сон, быть более энергичным, меньше болеть'}}
{'input_tokens': 1001, 'output_tokens': 46, 'conversation_tokens': 1047}


In [267]:
prompted_message = 'WEUASDRJG1873'
await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 8, 'output_tokens': 87, 'conversation_tokens': 1137}


USER:
 WEUASDRJG1873

\MODEL:
 Спасибо, Николай! Теперь у нас есть доступ к алгоритму. Давай перейдем к следующему вопросу. Какое у тебя артериальное давление? Какой тип давления у тебя: гипотония (менее 110/70), нормотония (110/70 - 130/90) или гипертония (более 130/90)?


In [268]:
prompted_message = 'мне понравились кубики у руслана, но не хватает заботы кураторов'
await send_message(prompted_message, message_history, usage_history)
readable_history = await generate_readable_history_from_end(message_history, 2)
response_new_info = await add_to_student_card(readable_history, extraction_history=[], usage_history=[])

{'input_tokens': 18, 'output_tokens': 87, 'conversation_tokens': 1242}


USER:
 мне понравились кубики у руслана, но не хватает заботы кураторов

\MODEL:
 Спасибо за отзыв, Николай! Я передам его нашей команде. Давай вернемся к твоему здоровью. Какое у тебя артериальное давление? Какой тип давления у тебя: гипотония (менее 110/70), нормотония (110/70 - 130/90) или гипертония (более 130/90)?


 NEW INFO {'Промокод': {'value': 'WEUASDRJG1873'}}
{'input_tokens': 1033, 'output_tokens': 23, 'conversation_tokens': 1056}


In [269]:
prompted_message = 'нормокинетический'
await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 5, 'output_tokens': 42, 'conversation_tokens': 1289}


USER:
 нормокинетический

\MODEL:
 Спасибо за информацию, Николай! А какой тип сокращения желчного пузыря у тебя? Возможные варианты: нормокинетический, гипо, гипер, неизвестно, желчный удален.


In [270]:
prompted_message = 'гипотоник'
await send_message(prompted_message, message_history, usage_history)
readable_history = await generate_readable_history_from_end(message_history, 2)
response_new_info = await add_to_student_card(readable_history, extraction_history=[], usage_history=[])

{'input_tokens': 4, 'output_tokens': 86, 'conversation_tokens': 1379}


USER:
 гипотоник

\MODEL:
 Спасибо, Николай! А какую желчегонную траву ты подобрал? Убедись, что выбранная трава соответствует типу твоего артериального давления. Гипертоникам подходят только кукурузные рыльца, которые уменьшают давление. Гипотоникам подходит только пижма, которая увеличивает давление, а нормотоникам подходит тысячелистник, который не изменяет давление.


 NEW INFO {'Артериальное давление': 'гипотоник', 'Тип сокращения желчного пузыря': 'нормокинетический'}
{'input_tokens': 980, 'output_tokens': 33, 'conversation_tokens': 1013}


In [271]:
prompted_message = 'какому уроку?'
await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 5, 'output_tokens': 45, 'conversation_tokens': 1429}


USER:
 какому уроку?

\MODEL:
 Я имею в виду урок по желчегонной гимнастике. У тебя есть камни в желчном пузыре? Если нет, я могу предложить тебе ссылку на урок по желчегонной гимнастике.


In [272]:
prompted_message = 'я выбрал тысячелистник'
await send_message(prompted_message, message_history, usage_history)
readable_history = await generate_readable_history_from_end(message_history, 2)
response_new_info = await add_to_student_card(readable_history, extraction_history=[], usage_history=[])

{'input_tokens': 7, 'output_tokens': 33, 'conversation_tokens': 1469}


USER:
 я выбрал тысячелистник

\MODEL:
 Хорошо, Николай! А сдавал ли ты анализы на уровень витамина Д в организме? В каких количествах планируешь принимать витамин Д?


 NEW INFO {'Трава желчегонная': {'value': 'тысячелистник'}}
{'input_tokens': 868, 'output_tokens': 23, 'conversation_tokens': 891}


In [273]:
prompted_message = 'да'
await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 1, 'output_tokens': 63, 'conversation_tokens': 1533}


USER:
 да

\MODEL:
 Замечательно, Николай! А в каких количествах ты планируешь принимать витамин Д? Если ты не уверен, я могу посоветовать тебе обратиться к нашему куратору Марии (@mariasmirnova03 в Telegram). Стандартная доза составляет 4000 МЕ.


In [274]:
prompted_message = '4000'
await send_message(prompted_message, message_history, usage_history)
readable_history = await generate_readable_history_from_end(message_history, 2)
response_new_info = await add_to_student_card(readable_history, extraction_history=[], usage_history=[])

{'input_tokens': 4, 'output_tokens': 24, 'conversation_tokens': 1561}


USER:
 4000

\MODEL:
 Отлично, Николай! А какое время для принятия лимфатического душа тебе подходит больше: утро или вечер?


 NEW INFO {'Имя и возраст': {'value': 'Николай, 29 лет'}, 'Уровень витамина Д в организме': {'value': 'да'}, 'Доза витамина Д': {'value': '4000'}}
{'input_tokens': 750, 'output_tokens': 55, 'conversation_tokens': 805}


In [275]:
prompted_message = 'днем'
await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 2, 'output_tokens': 27, 'conversation_tokens': 1590}


USER:
 днем

\MODEL:
 Николай, лимфатический душ обычно принимают утром или вечером, а не днем. В какое время тебе было бы удобнее?


In [276]:
prompted_message = 'утром'
await send_message(prompted_message, message_history, usage_history)
readable_history = await generate_readable_history_from_end(message_history, 2)
response_new_info = await add_to_student_card(readable_history, extraction_history=[], usage_history=[])

{'input_tokens': 2, 'output_tokens': 24, 'conversation_tokens': 1616}


USER:
 утром

\MODEL:
 Хорошо, Николай! А есть ли у тебя результаты копрограммы? Если да, можешь поделиться ими со мной?


 NEW INFO {'Время принятия лимфатического душа': {'value': 'утром'}}
{'input_tokens': 627, 'output_tokens': 22, 'conversation_tokens': 649}


In [277]:
prompted_message = 'нету'
await send_message(prompted_message, message_history, usage_history)

{'input_tokens': 2, 'output_tokens': 20, 'conversation_tokens': 1638}


USER:
 нету

\MODEL:
 Понятно, Николай. А есть ли у тебя какие-либо хронические заболевания или противопоказания?


In [278]:
prompted_message = 'да'
await send_message(prompted_message, message_history, usage_history)
readable_history = await generate_readable_history_from_end(message_history, 2)
response_new_info = await add_to_student_card(readable_history, extraction_history=[], usage_history=[])

{'input_tokens': 1, 'output_tokens': 29, 'conversation_tokens': 1668}


USER:
 да

\MODEL:
 Николай, можешь перечислить свои хронические заболевания или противопоказания? Это поможет нам составить более персонализированный алгоритм для тебя.


 NEW INFO {'Хронические заболевания': {'value': {}}}
{'input_tokens': 594, 'output_tokens': 16, 'conversation_tokens': 610}


In [283]:
prompted_message = 'у меня имеется анальная трещина'
await send_message(prompted_message, message_history, usage_history)

InvalidArgument: 400 Please ensure that multiturn requests ends with a user role or a function response.

In [285]:
student_card

{'description of card': 'карточка о здоровье, хранических заболеваниях, выбранном лечении',
 'Имя и возраст': {'value': 'Николай, 29 лет'},
 'Цели по здоровью': {'value': 'Наладить сон, быть более энергичным, меньше болеть'},
 'Промокод': {'value': 'WEUASDRJG1873'},
 'Впечатления о курсе': {'value': ''},
 'Артериальное давление': 'гипотоник',
 'Тип сокращения желчного пузыря': 'нормокинетический',
 'Трава желчегонная': {'value': 'тысячелистник'},
 'Желчегонная гимнастика': {'value': '',
  'instruction': 'Спросить о наличии камней в желчном пузыре, если их нет, то предложить выполнять желчегонную гимнастику. Урок по ссылке: https://goo.su/WlFpAPi'},
 'Уровень витамина Д в организме': {'value': 'да'},
 'Доза витамина Д': {'value': '4000'},
 'Время принятия лимфатического душа': {'value': 'утром'},
 'Результаты копрограммы': {'value': '',
  'instruction': 'Спросить о результатах копрограммы, если они имеются.'},
 'Хронические заболевания': {'value': {}}}

In [251]:
student_card

{'description of card': 'карточка о здоровье, хранических заболеваниях, выбранном лечении',
 'Имя и возраст': 'Николай, 29 лет',
 'Цели по здоровью': 'наладить сон, быть более энергичным, меньше болеть',
 'Промокод': 'WEUASDRJG1873',
 'Впечатления о курсе': '',
 'Артериальное давление': 'гипотоник',
 'Тип сокращения желчного пузыря': 'нормокинетический',
 'Трава желчегонная': 'тысячелистник',
 'Желчегонная гимнастика': '',
 'Доза витамина Д': '20',
 'Время принятия лимфатического душа': '',
 'Результаты копрограммы': ''}